In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import cv2
import sys
import math
import time
import random
import os
from datetime import datetime
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
from sewar.full_ref import vifp

In [4]:
from iEBMO import bmoIE

In [5]:
pSize = [30]
maxIter = 100
imageNameList = ['./data/DIPCO2017_dataset/1.bmp']
# imageNameList = ['a0027.tiff', 'a0037.tiff', 'a0050.tiff']
# imageNameList = ['a0027.png', 'a0037.png', 'a0050.png']

# print(imageNameList)

# imageNameList = ['./data/kodim/kodim01.png', './data/kodim/kodim02.png', './data/kodim/kodim03.png', './data/kodim/kodim04.png',
#                  './data/kodim/kodim05.png', './data/kodim/kodim06.png', './data/kodim/kodim07.png', './data/kodim/kodim08.png', './data/kodim/kodim09.png']


In [6]:
for i in range(len(imageNameList)):
    averagePsnr = 0
    averageSsim = 0
    averageVif = 0
    print(imageNameList[i])
    for popSize in pSize:
        print(popSize)

        # inputName = "GroundTruth/kodakDataset/"+imageNameList[i]
        truthName = imageNameList[i]

        trImg = cv2.imread(truthName, 0)
        cv2.imshow('input', trImg)
        histr = cv2.calcHist([trImg], [0], None, [256], [0, 256])
        plt.plot(histr)
        # histogram of ground truth
        # plt.savefig("./histogramsDIBCO/ht_"+imageNameList[i])
        plt.clf()
        # cv2.imwrite("./truthDIBCO/t_"+imageNameList[i], trImg)

        bestImage = deepcopy(trImg)

        maxPsnr = 0
        maxSsim = 0
        maxVif = 0
        maxfit = -1000
        for iteration in range(10):
            outputImage, fitval = bmoIE(truthName, popSize, maxIter)

            truthImage = cv2.imread(truthName, 0)
            psnrval = cv2.PSNR(outputImage, truthImage)
            ssimval = ssim(outputImage, truthImage)
            vifval = vifp(outputImage, truthImage)
            # print(iteration,psnrval,ssimval,vifval,int((psnrval+ssimval+vifval)*100))
#             print(psnrval, ssimval, vifval)
#             continue

            if (psnrval+ssimval+vifval) > maxfit:
                maxfit = psnrval+ssimval+vifval
                maxPsnr = psnrval
                maxSsim = ssimval
                maxVif = vifval
                bestImage = deepcopy(outputImage)
#             cv2.imshow(str(iteration+1),bestImage)

        print(maxPsnr, maxSsim, maxVif)

#         averagePsnr += maxPsnr
#         averageSsim += maxSsim
#         averageVif += maxVif
#         print(i+1,averagePsnr,averageSsim,averageVif)

#         histr = cv2.calcHist([bestImage],[0],None,[256],[0,256])
#         plt.plot(histr)
#         plt.savefig("histogramsDIBCO/ho_"+imageNameList[i]) #histogram of output image
#         plt.clf()
        transImageName = "o_"+imageNameList[i]
        cv2.imwrite("./outputDIBCO/"+transImageName, bestImage)


#     averagePsnr /= len(imageNameList)
#     averageSsim /= len(imageNameList)
#     averageVif /= len(imageNameList)
    print("final")
    print(averagePsnr)
    print(averageSsim)
    print(averageVif)

./data/DIPCO2017_dataset/1.bmp
30
8.208590490049661e+18
8.208590490049661e+18
6.226818734721026e+19
6.226818734721026e+19
4.899918101983945e+18
4.899918101983945e+18
1.0470942035696178e+20
1.0470942035696178e+20
1.0151194863511547e+20
1.0151194863511547e+20
2.402169186792806e+18
2.402169186792806e+18
1.3407250240104942e+19
1.3407250240104942e+19
1.7337384752492093e+19
1.7337384752492093e+19
3.797843474641302e+20
3.797843474641302e+20
4.001174287248216e+19
4.001174287248216e+19
2.027039407731568e+20
2.027039407731568e+20
1.0342849555913666e+20
1.0342849555913666e+20
1.4121052517704788e+20
1.4121052517704788e+20
3.451039154798479e+18
3.451039154798479e+18
1.5635453478820184e+20
1.5635453478820184e+20
1.5041889593683534e+19
1.5041889593683534e+19
1.6018309588775997e+19
1.6018309588775997e+19
2.014089682371066e+19
2.014089682371066e+19
4.96174258312124e+19
4.96174258312124e+19
1.4888742420991644e+19
1.4888742420991644e+19
4.112172272215874e+19
4.112172272215874e+19
7.450089382753329e+17
7.